In [1]:
import numpy as np 
import pandas as pd
import re
import gc
import os

import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report

/home/bharaj/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
def detect(tokens):
    return [t for t in tokens if t in valid_forms]
    
def replace_blank(tokens):
    return [blank if t in valid_forms else t for t in tokens]

def create_windows(tokens, window_size=3):
    X = []
    for i, word in enumerate(tokens):
        if word == blank:
            window = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
            window = ' '.join(window)
            X.append(window)    
    return X

In [4]:
f1 = open("../input/internet_archive_scifi_v3.txt","r")
a = f1.read()
a = re.sub('[\n]', '', a)
tokens = wordpunct_tokenize(a)
y = detect(tokens)
tokens = replace_blank(tokens)
X = create_windows(tokens)
f1.close()

In [5]:
df = pd.DataFrame()
df["Text"] = X
df["Label"] = y

In [6]:
df.head()

,Text,Label
0,Publisher Editor IF published bi -,is
1,in this magazine fiction . Any,are
2,to actual persons coincidental . #,is
3,. The title selected after much,was
4,the theory it indicative of the,is


In [7]:
one_hot = pd.get_dummies(df["Label"])
df.drop(['Label'],axis=1,inplace=True)
df = pd.concat([df,one_hot],axis=1)
df.head()

,Text,am,are,be,been,being,is,was,were
0,Publisher Editor IF published bi -,0,0,0,0,0,1,0,0
1,in this magazine fiction . Any,0,1,0,0,0,0,0,0
2,to actual persons coincidental . #,0,0,0,0,0,1,0,0
3,. The title selected after much,0,0,0,0,0,0,1,0
4,the theory it indicative of the,0,0,0,0,0,1,0,0


In [8]:
df1 = df[:200000]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1["Text"].values, df1.drop(['Text'],axis=1).values, test_size=0.2, random_state=42)

In [10]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=50)

In [11]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, maxlen=50)

In [12]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(8, activation='softmax'))


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train,
                    batch_size=1024,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Train on 144000 samples, validate on 16000 samples
Epoch 1/10
144000/144000 [==============================] - 9s 63us/step - loss: 1.2964 - acc: 0.5442 - val_loss: 0.8224 - val_acc: 0.6970
Epoch 2/10
144000/144000 [==============================] - 6s 42us/step - loss: 0.7231 - acc: 0.7306 - val_loss: 0.7121 - val_acc: 0.7288
Epoch 3/10
144000/144000 [==============================] - 6s 42us/step - loss: 0.6077 - acc: 0.7703 - val_loss: 0.6938 - val_acc: 0.7380
Epoch 4/10
144000/144000 [==============================] - 6s 42us/step - loss: 0.5328 - acc: 0.7999 - val_loss: 0.7015 - val_acc: 0.7392
Epoch 5/10
144000/144000 [==============================] - 6s 44us/step - loss: 0.4699 - acc: 0.8248 - val_loss: 0.7154 - val_acc: 0.7394
Epoch 6/10
144000/144000 [==============================] - 7s 47us/step - loss: 0.4172 - acc: 0.8458 - val_loss: 0.7408 - val_acc: 0.7341
Epoch 7/10
144000/144000 [==============================] - 6s 43us/step - loss: 0.3745 - acc: 0.8622 - val_loss: 0

In [15]:
score = model.evaluate(X_test, y_test,
                       batch_size=256, verbose=1)
print('Test accuracy:', score[1])

40000/40000 [==============================] - 2s 50us/step
Test accuracy: 0.722975


In [16]:
preds = model.predict(X_test)

In [17]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(preds,axis=1)))

              precision    recall  f1-score   support

           0       0.58      0.55      0.56       486
           1       0.55      0.50      0.53      2889
           2       0.92      0.95      0.93      6128
           3       0.95      0.93      0.94      3336
           4       0.41      0.32      0.36       765
           5       0.52      0.46      0.49      5911
           6       0.75      0.79      0.77     15788
           7       0.59      0.59      0.59      4697

   micro avg       0.72      0.72      0.72     40000
   macro avg       0.66      0.64      0.65     40000
weighted avg       0.72      0.72      0.72     40000



In [18]:
gc.collect()

0

# BERT implementation

In [6]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2020-01-13 18:00:08--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.193.128, 2a00:1450:400b:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.2’

uncased_L-12_H-768_ 100%[===================>] 388.84M  22.4MB/s    in 18s     

2020-01-13 18:00:26 (22.2 MB/s) - ‘uncased_L-12_H-768_A-12.zip.2’ saved [407727028/407727028]

--2020-01-13 18:00:27--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [

In [7]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip


--2020-01-13 18:01:03--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.193.128, 2a00:1450:400b:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip.1’

multi_cased_L-12_H- 100%[===================>] 632.19M  13.2MB/s    in 49s     

2020-01-13 18:01:53 (12.9 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip.1’ saved [662903077/662903077]



In [4]:
import modeling
import optimization
import run_classifier
import tokenization

W0115 09:01:18.860536 139863621216000 deprecation_wrapper.py:119] From /home/bharaj/papers/code-mix/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [5]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('whole_mal.tsv',sep='\t',names=['category','text'])
print(df.shape)

y=df['category']
df2 = pd.DataFrame()
df2["Text"] = df["text"]
df2["Label"] = LabelEncoder().fit_transform(y)
#df2 = df2[:200000]
print(df2.head())
X_train, X_test, y_train, y_test = train_test_split(df2["Text"].values, df2["Label"].values, test_size=0.2, random_state=5)

(6739, 2)
                                                Text  Label
0   Ichayan fans pinne mmade ettan fansm ivde oru...      4
1             Tovi 🥰 Best Wishes From #Kunjikka Fans      3
2   Urutty koll .nallavanaaya unniyaya saiju kuru...      4
3   Pls support me pls       My channel subscribe...      3
4     Kalki super hit akum enn Bonny parayan paranju      2


In [6]:
folder = 'model_folder'
with zipfile.ZipFile("multi_cased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)
BERT_MODEL = 'multi_cased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/multi_cased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs-n'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: model_folder/outputs-n
>>  BERT pretrained directory: model_folder/multi_cased_L-12_H-768_A-12


In [22]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: model_folder/outputs
>>  BERT pretrained directory: model_folder/uncased_L-12_H-768_A-12


In [13]:
# df2 = pd.DataFrame()
# df2["Text"] = df["text"]
# df2["Label"] = LabelEncoder().fit_transform(y)
# df2 = df2[:200000]
# df2.head()

,Text,Label
0,Ichayan fans pinne mmade ettan fansm ivde oru...,4
1,Tovi 🥰 Best Wishes From #Kunjikka Fans,3
2,Urutty koll .nallavanaaya unniyaya saiju kuru...,4
3,Pls support me pls My channel subscribe...,3
4,Kalki super hit akum enn Bonny parayan paranju,2


In [24]:
# X_train, X_test, y_train, y_test = train_test_split(df2["Text"].values, df2["Label"].values, test_size=0.2, random_state=42)

In [14]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 50
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(8)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

W0113 18:03:14.965779 140559672616704 deprecation_wrapper.py:119] From /home/bharaj/papers/code-mix/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0113 18:03:16.591481 140559672616704 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0113 18:03:16.594780 140559672616704 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fd605eef0d0>) includes params argument, but params are not passed to Estimator.
W0113 18:03:16.597678 140559672616704 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.


In [ ]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

W0113 18:03:43.242447 140559672616704 deprecation_wrapper.py:119] From /home/bharaj/papers/code-mix/run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



Please wait...


W0113 18:03:45.204365 140559672616704 deprecation.py:323] From /home/bharaj/anaconda3/lib/python3.6/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


>> Started training at 2020-01-13 18:03:45.185022 
  Num examples = 5391
  Batch size = 32


W0113 18:03:47.120745 140559672616704 deprecation_wrapper.py:119] From /home/bharaj/papers/code-mix/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0113 18:03:47.124726 140559672616704 deprecation_wrapper.py:119] From /home/bharaj/papers/code-mix/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0113 18:03:47.162734 140559672616704 deprecation_wrapper.py:119] From /home/bharaj/papers/code-mix/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0113 18:03:47.211796 140559672616704 deprecation.py:506] From /home/bharaj/papers/code-mix/modeling.py:358: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0113 18:03:47.235380 140559

In [ ]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [ ]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [ ]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy of BERT is:",accuracy_score(y_test,preds))

In [ ]:
print(classification_report(y_test,preds))